In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import scipy.optimize as opt
from tqdm import tqdm
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import lambdify

In [3]:
#filename = 'pareto_low_adv.csv'
#filename = 'pareto_low_sim.csv'
#filename = 'pareto_easy_adv.csv'
filename = 'pareto_easy_sim.csv'

In [4]:
t_eq=pd.read_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename)
len(t_eq)

288905

In [5]:
# drop rows that have the same value in column sympy_format.
t_eq=t_eq.drop_duplicates(subset='sympy_format',keep='first')
len(t_eq)

773

In [6]:
t_eq['fitting_format']=''
t_eq['num_fitting_variables']=0
t_eq['initial_constant_guess'] = pd.Series(dtype='object')

number_matching_pattern = r"(?<![a-zA-Z0-9_.])[+-]?(\d+\.\d+|\.\d+|\d+\.|\d+)(?:[eE][-+]?\d+)?"

for row in tqdm(t_eq.iterrows(),total=len(t_eq)):
    #equation = row[1]['sympy_format']
    #equation = parse_expr(equation)

    raw_equation = row[1]['equation']
    # replace the power sign
    raw_equation = raw_equation.replace('^','**')
    # evaluate the exp()
    equation = parse_expr(raw_equation).evalf()
    # swap log10(constant) with log(constant)/log(10)
    equation = re.sub(r'log10\((\d+\.\d+)\)',r'(log(\1)/log(10))',str(equation))
    # evaluate the log()
    equation = str(parse_expr(equation).evalf())


    # # swap constants
    # constants = re.findall(number_matching_pattern, equation)


    # variable_list=[]
    # for i in range(len(constants)):
    #     variable_list.append('p['+str(i)+']')


    # for i in range(len(constants)):
    #     equation = equation.replace(constants[i],variable_list[i])

    # swap constants v2
    # Find all unique matches first to avoid duplicates and incorrect indexing
    constants = list(set(re.findall(number_matching_pattern, equation)))
    # Sort constants by their length in descending order to replace longer numbers first, preventing partial replacement issues
    constants.sort(key=len, reverse=True)

    def replace_with_variable(match):
        # Find the matched number in the constants list and get its index
        number = match.group(1)
        index = constants.index(number)

        # Check if the match includes a minus sign
        if match.group(0).startswith('-'):
            sign = '-'
        else:
            sign = ''
        return f'{sign}p[{index}]'

    equation = re.sub(number_matching_pattern, replace_with_variable, equation)

    t_eq.loc[row[0],'fitting_format']=str(equation) 
    t_eq.loc[row[0],'num_fitting_variables']=len(constants)
    t_eq.at[row[0],'initial_constant_guess']=constants

100%|██████████| 773/773 [00:06<00:00, 119.91it/s]


In [7]:
t_eq=t_eq.drop_duplicates(subset='fitting_format',keep='last')
len(t_eq)

562

In [8]:
#t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/pareto_cleaned.csv',index=False)
t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename[:-4]+'_cleaned.csv',index=False)

In [9]:
t_eq

,complexity,loss,score,equation,sympy_format,lambda_format,number_constants,variables,number_variables,unique_number_variables,evolutions,iterations,fitting_format,num_fitting_variables,initial_constant_guess
0,1,2.422899,0.000000,x11,x11,PySRFunction(X=>x11),0,{'x11'},1,1,0,0,x11,0,[]
1,3,0.384125,0.920876,(x33 + 8.890797890812273),x33 + 8.890797890812273,PySRFunction(X=>x33 + 8.890797890812273),1,{'x33'},1,1,0,0,x33 + p[0],1,[8.89079789081227]
2,5,0.325642,0.082585,((x11 + -0.7475944403046133) + x13),x11 + x13 - 0.7475944403046133,PySRFunction(X=>x11 + x13 - 0.7475944403046133),1,"{'x11', 'x13'}",2,2,0,0,x11 + x13 - p[0],1,[0.747594440304613]
3,7,0.139009,0.425631,((x29 - x21) + (x11 + -1.5834719442805658)),x11 - x21 + x29 - 1.5834719442805658,PySRFunction(X=>x11 - x21 + x29 - 1.5834719442...,1,"{'x29', 'x21', 'x11'}",3,3,0,0,x11 - x21 + x29 - p[0],1,[1.58347194428057]
5,13,0.095671,0.007817,(((x3 * x3) + ((x3 - x21) + (0.429596908239727...,-x21 + x3**2 + x3 + 0.91903563791333875,PySRFunction(X=>-x21 + x3**2 + x3 + 0.91903563...,3,"{'x21', 'x3'}",4,2,0,0,-x21 + x3**p[1] + x3 + p[0],2,"[0.919035637913339, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259295,9,0.062104,0.046310,(x3 * (((x6 + 9.677901510292221) - x21) / 5.58...,x3*(-0.1789110412840841*x21 + 0.17891104128408...,PySRFunction(X=>x3*(-0.1789110412840841*x21 + ...,2,"{'x21', 'x6', 'x3'}",3,3,4,2722,x3*(-p[0]*x21 + p[0]*x6 + p[1]),2,"[0.178911041284084, 1.73148343665119]"
259397,5,0.080645,0.270588,((x3 * 0.33847654047787545) * x39),0.33847654047787545*x3*x39,PySRFunction(X=>0.33847654047787545*x3*x39),1,"{'x39', 'x3'}",2,2,4,2730,p[0]*x3*x39,1,[0.338476540477875]
262105,13,0.053479,0.042645,(x3 * (3.6315758373668716 - ((x21 - (x29 * x14...,x3*(3.6315758373668716 - (-x14*x29 + x21)/(x1*...,PySRFunction(X=>x3*(3.6315758373668716 - (-x14...,1,"{'x21', 'x14', 'x1', 'x5', 'x29', 'x3'}",6,6,4,2938,x3*(p[0] - (-x14*x29 + x21)/(x1*x5)),1,[3.63157583736687]
268226,9,0.060370,0.060472,((x3 * 0.16148889371776248) * ((x6 - x21) - -1...,0.16148889371776248*x3*(-x21 + x6 + 11.9570116...,PySRFunction(X=>0.16148889371776248*x3*(-x21 +...,2,"{'x21', 'x6', 'x3'}",3,3,4,3409,p[0]*x3*(-x21 + x6 + p[1]),2,"[0.161488893717762, 11.9570116040867]"


In [10]:
t_eq.iloc[5]

complexity                                                                15
loss                                                                0.090632
score                                                               0.027053
equation                   (((x3 * x3) + ((x3 - (x21 / x12)) + (0.4295969...
sympy_format                      x3**2 + x3 + 0.91903563791333875 - x21/x12
lambda_format              PySRFunction(X=>x3**2 + x3 + 0.919035637913338...
number_constants                                                           3
variables                                               {'x12', 'x21', 'x3'}
number_variables                                                           5
unique_number_variables                                                    3
evolutions                                                                 0
iterations                                                                 0
fitting_format                                x3**p[1] + x3 + p[0] - x21/x12

In [11]:
equation = 'x*exp(0.124)*log10(123.321)/log10(321.41+x2)*exp(log10(131.123))*log10(423.21+x3)'
# evaluate the exp()
equation = parse_expr(equation).evalf()
# swap log10(constant) with log(constant)/log(10)
equation = re.sub(r'log10\((\d+\.\d+)\)',r'(log(\1)/log(10))',str(equation))
# evaluate the log()
parse_expr(equation).evalf()

19.674807499434*x*log10(x3 + 423.21)/log10(x2 + 321.41)